# 📤 Predições no Conjunto de Teste — Submissão

## Objetivo
Gerar o arquivo de submissão para a competição aplicando o **melhor modelo treinado** (`model_0_xgboost`) sobre o conjunto de teste pré-processado.

## Pipeline
1. Carregar o vetorizador TF-IDF (Character N-grams) salvo no notebook 2.0
2. Carregar o modelo `model_0_xgboost.joblib` do diretório `char-ngrams-baseline`
3. Carregar os dados de teste limpos (`test_clean.csv`)
4. Vetorizar o texto de teste com o mesmo TF-IDF do treino
5. Gerar predições e salvar CSV no formato da competição

## Formato de saída
```
id,target
5398,1
5503,0
...
```


## 1. Setup

In [2]:
import json
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import joblib

MODEL_DIR = '../models/char-ngrams-baseline'

## 2. Carregar modelo e vetorizador TF-IDF

In [7]:
import os, joblib

PROCESSED_DIR = '../data/processed'

vect_path  = os.path.join(PROCESSED_DIR, 'tfidf_vectorizer.joblib')
model_path = os.path.join(MODEL_DIR, 'model_0_xgboost.joblib')

assert os.path.exists(vect_path),  f"Vetorizador não encontrado: {vect_path}"
assert os.path.exists(model_path), f"Modelo não encontrado: {model_path}"

vectorizer = joblib.load(vect_path)
model      = joblib.load(model_path)

print(f"✓ Vetorizador carregado  — {vect_path}")
print(f"  vocab: {len(vectorizer.vocabulary_):,} tokens | analyzer: {vectorizer.analyzer} | ngram: {vectorizer.ngram_range}")
print(f"\n✓ Modelo carregado       — {model_path}")
print(f"  tipo: {type(model).__name__}")


✓ Vetorizador carregado  — ../data/processed\tfidf_vectorizer.joblib
  vocab: 50,000 tokens | analyzer: char | ngram: (3, 5)

✓ Modelo carregado       — ../models/char-ngrams-baseline\model_0_xgboost.joblib
  tipo: XGBClassifier


## 3. Carregar dados de teste

- **IDs** vêm do `test.csv` (raw)
- **Texto limpo** vem do `test_clean.csv` (pré-processado no notebook 2.0)

In [8]:
# IDs do arquivo raw
df_raw = pd.read_csv('../data/raw/test.csv')
ids = df_raw['id'].values

# Texto limpo do pré-processamento
df_clean = pd.read_csv('../data/processed/test_clean.csv')

# Preencher nulos e combinar title + text (mesmo pipeline do treino)
df_clean['title'] = df_clean['title'].fillna('')
df_clean['text'] = df_clean['text'].fillna('')
df_clean['full_text'] = df_clean['title'] + ' ' + df_clean['text']

assert len(ids) == len(df_clean), f"Mismatch: {len(ids)} IDs vs {len(df_clean)} textos"

print(f"Amostras de teste: {len(df_clean):,}")

Amostras de teste: 5,712


## 4. Vetorizar e gerar predições

In [9]:
import time

print("Vetorizando conjunto de teste...")
t0 = time.time()
X_test = vectorizer.transform(df_clean['full_text'])
elapsed = time.time() - t0

print(f"✓ Vetorização concluída em {elapsed:.1f}s")
print(f"  Shape: {X_test.shape[0]:,} docs × {X_test.shape[1]:,} features")

print("\nGerando predições com XGBoost...")
t0 = time.time()
preds = model.predict(X_test)
elapsed = time.time() - t0

print(f"✓ Predições geradas em {elapsed:.1f}s")
print(f"\nDistribuição das predições:")
unique, counts = np.unique(preds, return_counts=True)
for cls, cnt in zip(unique, counts):
    label = 'Fake' if cls == 1 else 'Real'
    print(f"  Classe {int(cls)} ({label}): {cnt:,} ({cnt / len(preds):.1%})")


Vetorizando conjunto de teste...
✓ Vetorização concluída em 20.2s
  Shape: 5,712 docs × 50,000 features

Gerando predições com XGBoost...
✓ Predições geradas em 0.2s

Distribuição das predições:
  Classe 0 (Real): 4,281 (74.9%)
  Classe 1 (Fake): 1,431 (25.1%)


## 5. Salvar CSV de submissão

In [10]:
SUBMISSION_PATH = '../data/processed/submission_xgboost.csv'

submission = pd.DataFrame({
    'id':     ids,
    'target': preds.astype(int),
})

submission.to_csv(SUBMISSION_PATH, index=False)

print(f"✓ Submissão salva em: {SUBMISSION_PATH}")
print(f"  Linhas: {len(submission):,}")
print(f"\nPrimeiras 10 linhas:")
print(submission.head(10).to_string(index=False))


✓ Submissão salva em: ../data/processed/submission_xgboost.csv
  Linhas: 5,712

Primeiras 10 linhas:
   id  target
 5398       1
 5503       1
23151       0
12669       0
27864       0
23035       0
25746       0
27194       0
  933       1
11610       0


## 6. Validação do formato

In [11]:
df_sub = pd.read_csv(SUBMISSION_PATH)

checks = {
    'Colunas corretas (id, target)'  : list(df_sub.columns) == ['id', 'target'],
    'Sem valores nulos'              : df_sub.isnull().sum().sum() == 0,
    'Apenas classes 0 e 1'          : set(df_sub['target'].unique()).issubset({0, 1}),
    'IDs únicos'                     : df_sub['id'].nunique() == len(df_sub),
    f'Nº de linhas = {len(ids):,}'   : len(df_sub) == len(ids),
}

print("=" * 50)
print("VALIDAÇÃO DO ARQUIVO DE SUBMISSÃO")
print("=" * 50)
all_ok = True
for check, result in checks.items():
    status = '✓' if result else '✗ FALHOU'
    if not result:
        all_ok = False
    print(f"  {status}  {check}")

print()
if all_ok:
    print("✅ Arquivo válido para submissão!")
    print(f"   Arquivo : {SUBMISSION_PATH}")
    print(f"   Modelo  : model_0_xgboost  ({type(model).__name__})")
    print(f"   Amostras: {len(df_sub):,}")
else:
    print("❌ Corrija os erros antes de submeter.")


VALIDAÇÃO DO ARQUIVO DE SUBMISSÃO
  ✓  Colunas corretas (id, target)
  ✓  Sem valores nulos
  ✓  Apenas classes 0 e 1
  ✓  IDs únicos
  ✓  Nº de linhas = 5,712

✅ Arquivo válido para submissão!
   Arquivo : ../data/processed/submission_xgboost.csv
   Modelo  : model_0_xgboost  (XGBClassifier)
   Amostras: 5,712


## 7. Comparação de Submissões — ExtraTrees vs XGBoost

Como ambos os modelos obtiveram o **mesmo score** na competição, analisamos:
- Taxa de concordância entre as predições
- Exemplos de **discordâncias** (onde um modelo acerta e o outro pode errar)
- **Ensemble por votação majoritária** (ambos concordam -> usa a predição; discordam -> define regra de desempate)


In [12]:

# ── Carregar as duas submissões ───────────────────────────────────────────────
sub_et  = pd.read_csv('../data/processed/submission_extratrees.csv').rename(columns={'target': 'extratrees'})
sub_xgb = pd.read_csv('../data/processed/submission_xgboost.csv').rename(columns={'target': 'xgboost'})

df_cmp = sub_et.merge(sub_xgb, on='id')

# ── Concordância global ───────────────────────────────────────────────────────
agree_mask  = df_cmp['extratrees'] == df_cmp['xgboost']
n_total     = len(df_cmp)
n_agree     = agree_mask.sum()
n_disagree  = n_total - n_agree

print("=" * 60)
print("COMPARAÇÃO: ExtraTrees vs XGBoost")
print("=" * 60)
print(f"  Total de amostras  : {n_total:,}")
print(f"  Concordam          : {n_agree:,}  ({n_agree/n_total:.2%})")
print(f"  Discordam          : {n_disagree:,}  ({n_disagree/n_total:.2%})")

# ── Detalhamento das discordâncias ────────────────────────────────────────────
df_disagree = df_cmp[~agree_mask].copy()
if n_disagree > 0:
    et1_xgb0 = ((df_disagree['extratrees'] == 1) & (df_disagree['xgboost'] == 0)).sum()
    et0_xgb1 = ((df_disagree['extratrees'] == 0) & (df_disagree['xgboost'] == 1)).sum()
    print(f"\n  Discordâncias por padrão:")
    print(f"    ET=Fake / XGB=Real : {et1_xgb0:,}")
    print(f"    ET=Real / XGB=Fake : {et0_xgb1:,}")

    print(f"\n  Primeiras 10 discordâncias:")
    print(df_disagree.head(10).to_string(index=False))

# ── Distribuição das predições por modelo ─────────────────────────────────────
print(f"\n{'─'*60}")
print(f"{'Classe':<12} {'ExtraTrees':>12} {'XGBoost':>12}")
print(f"{'─'*60}")
for cls in [0, 1]:
    label = 'Real (0)' if cls == 0 else 'Fake (1)'
    n_et  = (df_cmp['extratrees'] == cls).sum()
    n_xgb = (df_cmp['xgboost']   == cls).sum()
    print(f"{label:<12} {n_et:>9,} ({n_et/n_total:.1%})  {n_xgb:>9,} ({n_xgb/n_total:.1%})")


COMPARAÇÃO: ExtraTrees vs XGBoost
  Total de amostras  : 5,712
  Concordam          : 5,710  (99.96%)
  Discordam          : 2  (0.04%)

  Discordâncias por padrão:
    ET=Fake / XGB=Real : 1
    ET=Real / XGB=Fake : 1

  Primeiras 10 discordâncias:
   id  extratrees  xgboost
22809           1        0
16447           0        1

────────────────────────────────────────────────────────────
Classe         ExtraTrees      XGBoost
────────────────────────────────────────────────────────────
Real (0)         4,281 (74.9%)      4,281 (74.9%)
Fake (1)         1,431 (25.1%)      1,431 (25.1%)
